## 1) Importação dos packages necessários

In [1]:
import requests
import ast
import pandas as pd
import time
from urllib.error import URLError, HTTPError
from urllib.request import Request, urlopen
import bs4

## 2) Retirar a informação da página web

In [2]:
# Informação do agente

agente = "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Mobile Safari/537.36"
headers = {'User-Agent': agente}

In [3]:
# Função para chamar a página WEB

def ConsultaWebB(url):
    try:
        req = Request(url, headers = headers)
        response = urlopen(req)
        return response.read()

    except:
        pass

In [4]:
#teste à função:
#ConsultaWebB('https://www.legislador.com.br//LegisladorWEB.ASP?WCI=ProposicaoTexto&ID=3&TPProposicao=1&nrProposicao=200&aaProposicao=2021')

In [5]:
# Função para transformar a página encontrada em um objeto utilizável

def captura_html_pagina(url):
    html = ConsultaWebB(url)
    soup = bs4.BeautifulSoup(html, 'html.parser')    
    return soup

In [6]:
#teste
#captura_html_pagina('https://www.legislador.com.br//LegisladorWEB.ASP?WCI=ProposicaoTexto&ID=3&TPProposicao=1&nrProposicao=200&aaProposicao=2021')

In [7]:
# Função para retirar as informações do cabeçalho da página

def Cabecalho(html):
    dt = html.find_all('dt')
    dd = html.find_all('dd')
    dic = {}
    for i in range(len(dt)):
        x = dt[i].get_text()
        y = dd[i].get_text() 
        dic[x] = y
    return dic

In [8]:
#teste
#url='https://www.legislador.com.br//LegisladorWEB.ASP?WCI=ProposicaoTexto&ID=3&TPProposicao=1&nrProposicao=200&aaProposicao=2021'
#html = captura_html_pagina(url)
#Cabecalho(html)

In [9]:
# Funçao para retirar o conteúdo da página e transformar em um dicionário
def Conteudo(proposicao, ano):
    url='https://www.legislador.com.br//LegisladorWEB.ASP?WCI=ProposicaoTexto&ID=3&TPProposicao=1&nrProposicao='+str(proposicao)+'&aaProposicao='+str(ano)
    html = captura_html_pagina(url) # função para transformar a página encontrada em um objeto utilizável - utiliza o beautiful soup
    dic = Cabecalho(html) # função que retira a informação do cabeçalho e coloca num dicionário - variáveies: Reunião, Deliberação, Situação, Assunto e Autor (utilizam os mesmos marcadores - dt e dd)
    dic['Proposição'] = proposicao #acrescento no dicionário informação do n.º da proposição - informação no url e input da função
    dic['Ano'] = ano #acrescento no dicionário informação do ano -informação do url e input da função
    dic['Texto'] = html.p.get_text() #acrescento no dicionário informação do texto (utiliza o marcador p)
    return dic

In [10]:
#Teste
Conteudo(200,2021)

{'Reunião': '11/02/2021',
 'Deliberação': '11/02/2021',
 'Situação': 'Proposição Aprovada',
 'Assunto': 'Manutenção de via pública',
 'Autor': 'VereadorValentim Blasius.',
 'Proposição': 200,
 'Ano': 2021,
 'Texto': 'O Vereador abaixo firmado requer, após ouvido o Plenário, encaminhamento de cópia da presente Indicação à Secretaria de Urbanização e Meio Ambiente solicitando manutenção de calçada da rua 25 de Maio, defronte ao nº 266, Carijós. A pedido de moradores.'}

In [11]:
#Função final - vou criar uma função que coloca a informação de cada página num dicionário, depois acrescento cada dicionário numa lista e no final transformo num dataframe
# utilização do try except para evitar os erros no url (por exemplo não existirem)

def TabelaResultados(inicar_em, quantidade, ano, erros_admissiveis, segundos_espera):

    ultima_consulta = inicar_em + quantidade - 1

    # erros
    erros = 0

    # variaveis para loop
    i = 1
    lista = []

    while inicar_em <= ultima_consulta and erros <= erros_admissiveis:

        try:
            x = Conteudo(inicar_em,ano)
            lista = lista + [Conteudo(inicar_em,ano)]
        except:
            erros += 1
            pass

        time.sleep(segundos_espera)

        # carregamento incremental das variáveis
        inicar_em+=1 
        i+=1


    return pd.DataFrame(lista)

In [12]:
#teste
teste = TabelaResultados(401,10,2021,2,0.2)
teste

,Ano,Assunto,Autor,Deliberação,Proposição,Reunião,Situação,Texto
0,2021,"Limpeza, Macadamização, Patrolamento, Retifica...",VereadorRoger Michel Knipers.,15/03/2021,401,15/03/2021,Proposição Aprovada,"O vereador abaixo firmado requer, após ouvido ..."
1,2021,Manutenção de via pública,VereadorAna Paula Reiter.,15/03/2021,402,15/03/2021,Proposição Aprovada,"A vereadora abaixo firmada requer, após ouvido..."
2,2021,Manutenção de via pública,VereadorJonas Luiz de Lima.,NaN,403,18/03/2021,Entrada na Ordem do Dia,"O Vereador abaixo firmado requer, após ouvido ..."
3,2021,"Limpeza, Macadamização, Patrolamento, Retifica...",VereadorFlávio Augusto Ferri Molinari.,NaN,404,18/03/2021,Entrada na Ordem do Dia,"O vereador que está subscreve, no uso das atri..."
4,2021,"Limpeza, Macadamização, Patrolamento, Retifica...",VereadorFlávio Augusto Ferri Molinari.,NaN,405,18/03/2021,Entrada na Ordem do Dia,"O vereador que está subscreve, no uso das atri..."
5,2021,Manutenção de via pública,VereadorFlávio Augusto Ferri Molinari.,NaN,406,18/03/2021,Entrada na Ordem do Dia,"O vereador que está subscreve, no uso das atri..."
6,2021,"Limpeza, Macadamização, Patrolamento, Retifica...",VereadorDiogo de Pinho.,NaN,407,18/03/2021,Entrada na Ordem do Dia,"O vereador que esta subscreve, no uso das atri..."
7,2021,Manutenção de via pública,VereadorFernanda dos Santos Cattoni.,NaN,408,18/03/2021,Entrada na Ordem do Dia,"A vereadora abaixo firmada requer, após ouvido..."
8,2021,"Pontes, Bueiros, boca de lobo",VereadorFernanda dos Santos Cattoni.,NaN,409,18/03/2021,Entrada na Ordem do Dia,"A vereadora que esta subscreve, no uso das atr..."
9,2021,Manutenção de via pública,VereadorFernanda dos Santos Cattoni.,NaN,410,18/03/2021,Entrada na Ordem do Dia,"A vereadora que esta subscreve, no uso das atr..."


## 2) Criar a base de dados (sqlite3) e tabela

### 2.1) Criar a base de dados utilizando o sqlite3 e a tabela proposições

### 2.2) Alternativa - criar através do código

In [13]:

import sqlite3 #package sqlite
conn = sqlite3.connect(r"C:\Users\fiu126\Desktop\Coursera\Maratona Power BI e Python\alternativa.sqlite") #cria o ficheiro sql - neste caso sqlite
cur = conn.cursor() #uma espécie de handle

#criar  a tabela proposições - primiero elimino se existir e depois crio a nova tabela
cur.executescript('''

DROP TABLE IF EXISTS Proposicoes;

CREATE TABLE Proposicoes (
    DataReuniao DATE,
    DataDeliberacao DATE,
    Situacao VARCHAR(200),
    Assunto VARCHAR(1000),
    Autor VARCHAR(1000),
    Proposicao INT,
    Ano INT,
    Texto VARCHAR(8000)
)
''')

### Fim 2.1 e 2.2

In [14]:
#package e ligação à base de dados
import sqlite3 #package sqlite
#conn = sqlite3.connect(r"C:\Users\fiu126\Desktop\Coursera\Maratona Power BI e Python\Indaial.db")
conn = sqlite3.connect(r"C:\Users\fiu126\Desktop\Coursera\Maratona Power BI e Python\alternativa.sqlite")
cur = conn.cursor() #uma espécie de handle

In [15]:
#teste de consulta
query='''
select *
from Proposicoes
limit 10
'''
#Utilizo o panda para executar a query - query e conn (conexão)
sql_query=pd.read_sql_query(query,conn)
sql_query

,DataReuniao,DataDeliberacao,Situacao,Assunto,Autor,Proposicao,Ano,Texto


In [16]:
# Criamos uma base (estou a criar um DataFrame com as colunas todas que quero vazio de modo a ter sempre as colunas todas)
base = pd.DataFrame(columns=['Reunião', 'Deliberação', 'Situação', 'Assunto', 'Autor', 'Proposição', 'Ano', 'Texto'])
base

,Reunião,Deliberação,Situação,Assunto,Autor,Proposição,Ano,Texto


In [17]:
# função personalizada para fazer querys

def SQLSelect(query):
    
    #conn = sqlite3.connect(r"C:\Users\fiu126\Desktop\Coursera\Maratona Power BI e Python\Indaial.db")
    conn = sqlite3.connect(r"C:\Users\fiu126\Desktop\Coursera\Maratona Power BI e Python\alternativa.sqlite")

    out = pd.read_sql_query(query,conn)
    return out

In [18]:
SQLSelect('''
    select *
    from Proposicoes
    limit 5
''')

,DataReuniao,DataDeliberacao,Situacao,Assunto,Autor,Proposicao,Ano,Texto


In [19]:
#colocar a inserção de dados numa função
def SQLInsertProposicoes(TabelaProposicoes):
    
    base = pd.DataFrame(columns=['Reunião', 'Deliberação', 'Situação', 'Assunto', 'Autor', 'Proposição', 'Ano', 'Texto']) #cria a tabela de base - DataFrame (importante porque alguns )
    TabelaProposicoes = base.append(TabelaProposicoes).fillna('')

    #conn = sqlite3.connect(r"C:\Users\fiu126\Desktop\Coursera\Maratona Power BI e Python\Indaial.db")
    conn = sqlite3.connect(r"C:\Users\fiu126\Desktop\Coursera\Maratona Power BI e Python\alternativa.sqlite")

    cursor = conn.cursor()

    for index, row in TabelaProposicoes.iterrows():

        cursor.execute('''

            INSERT INTO Proposicoes (
                DataReuniao,
                DataDeliberacao,
                Situacao,
                Assunto,
                Autor,
                Proposicao,
                Ano,
                Texto
            ) 
            values(?,?,?,?,?,?,?,?)''', 

            [row['Reunião'], 
            row['Deliberação'], 
            row['Situação'], 
            row['Assunto'], 
            row['Autor'], 
            row['Proposição'], 
            row['Ano'], 
            row['Texto']]

        )

    conn.commit()
    cursor.close()

In [20]:
#teste
SQLInsertProposicoes(teste)
SQLSelect('''
    select *
    from Proposicoes
    limit 5
''')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,DataReuniao,DataDeliberacao,Situacao,Assunto,Autor,Proposicao,Ano,Texto
0,15/03/2021,15/03/2021,Proposição Aprovada,"Limpeza, Macadamização, Patrolamento, Retifica...",VereadorRoger Michel Knipers.,401,2021,"O vereador abaixo firmado requer, após ouvido ..."
1,15/03/2021,15/03/2021,Proposição Aprovada,Manutenção de via pública,VereadorAna Paula Reiter.,402,2021,"A vereadora abaixo firmada requer, após ouvido..."
2,18/03/2021,,Entrada na Ordem do Dia,Manutenção de via pública,VereadorJonas Luiz de Lima.,403,2021,"O Vereador abaixo firmado requer, após ouvido ..."
3,18/03/2021,,Entrada na Ordem do Dia,"Limpeza, Macadamização, Patrolamento, Retifica...",VereadorFlávio Augusto Ferri Molinari.,404,2021,"O vereador que está subscreve, no uso das atri..."
4,18/03/2021,,Entrada na Ordem do Dia,"Limpeza, Macadamização, Patrolamento, Retifica...",VereadorFlávio Augusto Ferri Molinari.,405,2021,"O vereador que está subscreve, no uso das atri..."


In [21]:
# Estou a utilizar um f e {} - código para truncar a tabela

#The TRUNCATE T#ABLE statement is used to remove all records from a table.

#SQLite does not have an explicit TRUNCATE TABLE command like other databases. 
#Instead, it has added a TRUNCATE optimizer to the DELETE statement. To truncate a table in SQLite, you just need to execute a DELETE statement without a WHERE clause. 
#The TRUNCATE optimizer handles the rest. Let's explain.

def SQLTruncate(NomeTabela):

    #conn = sqlite3.connect(r"C:\Users\fiu126\Desktop\Coursera\Maratona Power BI e Python\Indaial.db")
    conn = sqlite3.connect(r"C:\Users\fiu126\Desktop\Coursera\Maratona Power BI e Python\alternativa.sqlite")


    cursor = conn.cursor()

    cursor.execute(f'''

                   DELETE FROM {NomeTabela}

                   ''')

    conn.commit()
    cursor.close()

In [22]:
# Limpando dados da tabela
SQLTruncate('Proposicoes')

# Visualizando a tabela
SQLSelect('select *from Proposicoes')

,DataReuniao,DataDeliberacao,Situacao,Assunto,Autor,Proposicao,Ano,Texto


## 3) Alimentação Incremental

In [23]:
# ========= INICIO =========
ano = 2021

dados_ano = SQLSelect(f'select max(Proposicao) as Proposicao from Proposicoes where Ano = {ano}') #query ao máximo de proposição - saber qual foi a última informação carregada
ultima_proposicao = dados_ano['Proposicao'].loc[0] # retiro o valor


#condição que garante se o resultado for nulo ele considera 1
# ----- INICIO ALTERAÇÃO ----- 
if ultima_proposicao == None: #'None' - plalavra reservada no python
    proxima_proposicao = 1
else:
    proxima_proposicao = int(ultima_proposicao) + 1 
# ----- FIM ALTERAÇÃO ----- 


dados = Conteudo(proxima_proposicao,ano)
tabela = pd.DataFrame([dados])
SQLInsertProposicoes(tabela)
# ========= FIM =========


# visualizando dados
SQLSelect('select * from Proposicoes')

,DataReuniao,DataDeliberacao,Situacao,Assunto,Autor,Proposicao,Ano,Texto
0,01/02/2021,01/02/2021,Proposição Aprovada,Trânsito,VereadorRemir José de Faveri.,1,2021,"O vereador que esta subscreve, no uso das atri..."


In [24]:
#criar uma função

def InsereProximaProposicao(ano):

    # identifica  a última proposição registada 
    dados_ano = SQLSelect(f'select max(Proposicao) as Proposicao  from Proposicoes where Ano = {ano}')
    ultima_proposicao = dados_ano['Proposicao'].loc[0]

    # Verifica se foi identificado qualquer lançamento naquele ano
    if ultima_proposicao == None:
        proxima_proposicao = 1
    else:
        proxima_proposicao = int(ultima_proposicao) + 1 

    # Captura e Insere dados na tabela
    dados = Conteudo(proxima_proposicao,ano)
    tabela = pd.DataFrame([dados])
    SQLInsertProposicoes(tabela)

In [25]:
#teste
InsereProximaProposicao(2021)
# visualizando dados
SQLSelect('select * from Proposicoes')

,DataReuniao,DataDeliberacao,Situacao,Assunto,Autor,Proposicao,Ano,Texto
0,01/02/2021,01/02/2021,Proposição Aprovada,Trânsito,VereadorRemir José de Faveri.,1,2021,"O vereador que esta subscreve, no uso das atri..."
1,01/02/2021,01/02/2021,Proposição Aprovada,Manutenção de via pública,VereadorRemir José de Faveri.,2,2021,"O vereador abaixo firmado requer, após ouvido ..."


In [26]:
# limpar a informação da tabela
SQLTruncate('Proposicoes')

# Visualizar o resultado
SQLSelect('select *from Proposicoes')

,DataReuniao,DataDeliberacao,Situacao,Assunto,Autor,Proposicao,Ano,Texto


In [27]:
#importação da informação de um ano específico - por exemplo 1996 (a condição de paragem é enquanto não existir dois erros)

ano = 1996
quantidade = 999999
erros_admissiveis = 2
segundos_espera = 0

# erros
erros = 0

# variaveis para loop
i = 1
lista = []

while erros <= erros_admissiveis:

    try:
        InsereProximaProposicao(ano)
    except:
        erros += 1
        pass

    time.sleep(segundos_espera)

    # carregamento incremental das variáveis
    i +=1

In [28]:
# Visualizar o resultado - 1996 tem 24 proposições
SQLSelect('select count(*) from Proposicoes')

,count(*)
0,24


In [29]:
#colocar os parametros numa função
def BuscaGravaDadosAno(ano, quantidade = 10, erros_admissiveis = 2, segundos_espera = 0):

    # erros
    erros = 0

    # variaveis para loop
    i = 1
    lista = []

    while erros <= erros_admissiveis:

        try:
            InsereProximaProposicao(ano)
        except:
            erros += 1
            pass

        time.sleep(segundos_espera)

        # carregamento incremental das variáveis
        i +=1

In [30]:
BuscaGravaDadosAno(1996,9999,2,1)
SQLSelect('select count(*) from Proposicoes')

,count(*)
0,24


In [31]:
# Limpando dados da tabela
SQLTruncate('Proposicoes')

# Visualizando a tabela
SQLSelect('select *from Proposicoes')

,DataReuniao,DataDeliberacao,Situacao,Assunto,Autor,Proposicao,Ano,Texto


In [32]:
# colocar num for loop - retirar toda a informação - 1996 a 2021
ano_inicial = 2020
ano_final = 2021

for i in list(range(ano_inicial, ano_final+1)):
    print('Iniciando gravação dos dados do ano: ',i)
    try:
        BuscaGravaDadosAno(i, quantidade = 999999)
    except:
        pass
    
print('Inserção finalizada 😁')

Iniciando gravação dos dados do ano:  2020
Iniciando gravação dos dados do ano:  2021
Inserção finalizada 😁
